# Opcode Sequence을 사용한 machineLearning 기반 malware detection Model
- dataset: https://github.com/iosifache/DikeDataset
- Accuracy: 0.9852602487333026

# 필요 package 설치

In [1]:
!pip install pefile
!pip install capstone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 509 kB/s 
  Created wheel for pefile: filename=pefile-2021.9.3-py3-none-any.whl size=68844 sha256=93d53c1c53243da3558c8ec6f280043aaebee6fd0e07d875c85249d798248dd7
  Stored in directory: /root/.cache/pip/wheels/50/be/9b/d19a6151b9c6e303f823faedf03193d7e90dac4009a0bdafeb
Successfully built pefile
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.0 MB/s 


# utils.py 파일, sample 파일이 있는 폴더로 위치 변경

In [2]:
%cd /content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test

/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test


# 필요 package Import

In [3]:
from utils import *

from xgboost import XGBClassifier

## opcode 리스트 로드
- file 폴더에 있는 opcodesList.txt 파일을 filePath로 설정하시면 됩니다.

In [4]:
filePath = '/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/files/opcode database/opcodesList.txt'
with open(filePath, 'rb') as lf:
    opcodes = pickle.load(lf)
    print(opcodes)

['aaa', 'aad', 'aam', 'aas', 'adc', 'add', 'and', 'call', 'cbw', 'clc', 'cld', 'cli', 'cmc', 'cmp', 'cmpsb', 'cmpsw', 'cwd', 'daa', 'das', 'dec', 'div', 'esc', 'hlt', 'idiv', 'imul', 'in', 'inc', 'int', 'into', 'iret', 'jcc', 'ja', 'jae', 'jb', 'jbe', 'jc', 'je', 'jg', 'jge', 'jl', 'jle', 'jnle', 'jno', 'jnp', 'jns', 'jnz', 'jo', 'jp', 'jpe', 'jpo', 'js', 'jz', 'jcxz', 'jmp', 'lahf', 'lds', 'lea', 'les', 'lock', 'lodsb', 'lodsw', 'loop', 'loope', 'loopne', 'loopnz', 'loopz', 'loopx', 'mov', 'movsb', 'movsw', 'mul', 'neg', 'nop', 'not', 'or', 'out', 'pop', 'popf', 'push', 'pushf', 'rcl', 'rcr', 'repxx', 'rep', 'repe', 'repne', 'repnz', 'repz', 'ret', 'retn', 'retf', 'rol', 'ror', 'sahf', 'sal', 'sar', 'sbb', 'scasb', 'scasw', 'shl', 'shr', 'stc', 'std', 'sti', 'stosb', 'stosw', 'sub', 'test', 'wait', 'xchg', 'xlat', 'xor']


In [5]:
opcodeDict = {}
for i in range(len(opcodes)):
    opcodeDict.setdefault(opcodes[i], i)

print(opcodeDict)

{'aaa': 0, 'aad': 1, 'aam': 2, 'aas': 3, 'adc': 4, 'add': 5, 'and': 6, 'call': 7, 'cbw': 8, 'clc': 9, 'cld': 10, 'cli': 11, 'cmc': 12, 'cmp': 13, 'cmpsb': 14, 'cmpsw': 15, 'cwd': 16, 'daa': 17, 'das': 18, 'dec': 19, 'div': 20, 'esc': 21, 'hlt': 22, 'idiv': 23, 'imul': 24, 'in': 25, 'inc': 26, 'int': 27, 'into': 28, 'iret': 29, 'jcc': 30, 'ja': 31, 'jae': 32, 'jb': 33, 'jbe': 34, 'jc': 35, 'je': 36, 'jg': 37, 'jge': 38, 'jl': 39, 'jle': 40, 'jnle': 41, 'jno': 42, 'jnp': 43, 'jns': 44, 'jnz': 45, 'jo': 46, 'jp': 47, 'jpe': 48, 'jpo': 49, 'js': 50, 'jz': 51, 'jcxz': 52, 'jmp': 53, 'lahf': 54, 'lds': 55, 'lea': 56, 'les': 57, 'lock': 58, 'lodsb': 59, 'lodsw': 60, 'loop': 61, 'loope': 62, 'loopne': 63, 'loopnz': 64, 'loopz': 65, 'loopx': 66, 'mov': 67, 'movsb': 68, 'movsw': 69, 'mul': 70, 'neg': 71, 'nop': 72, 'not': 73, 'or': 74, 'out': 75, 'pop': 76, 'popf': 77, 'push': 78, 'pushf': 79, 'rcl': 80, 'rcr': 81, 'repxx': 82, 'rep': 83, 'repe': 84, 'repne': 85, 'repnz': 86, 'repz': 87, 'ret': 

# 학습된 XGBClassifier 모델 로드

# 모델 로드

In [6]:
# 파일명
filename = 'XGBClassifier.model'

# 모델 불러오기
clf = pickle.load(open(filename, 'rb'))

## MalwareDetectionFunction을 통한 Malware 탐지
- 양성 샘플 파일이 양성으로 결과가 나오는지 확인
- return 1: 악성코드, return 0: 양성코드

In [9]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunction(model, fileName):
    try:
        #parse exe file
        exe = pefile.PE(fileName)
        #call the function we created earlier
        sampleOpcodeList = ExtractPefileOpcodes(opcodes, exe)
    except:
        print('Error File')

    sampleFrequencyList = [0 for i in range(len(opcodes))]
    for op in sampleOpcodeList:
        sampleFrequencyList[opcodeDict[op]] += 1

    dict_list = [[] for o in opcodes]

    for i in range(len(opcodes)):
        dict_list[i].append(sampleFrequencyList[i])

    dictionary = dict(zip(opcodes, dict_list))
    sampleDf = pd.DataFrame.from_dict(dictionary)
    sampleDf.head()

    return model.predict(sampleDf)[0]

- 결과가 0으로 양성파일을 탐지한 것읋 확인할 수 있다.

In [10]:
fileName = '3f3fe9ecad7f30fc80cdfb678d7ca27a30d0575a73818746e98be9170d3be348.exe'

MalwareDetectionFunction(clf, fileName)

0

# opcode Sequence List pickle을 입력으로 사용하는 탐지 함수
- 악성코드 파일을 직접 Google Drive 및 GitHub에 올릴 수 없기 때문에 pickle로 먼저 opcodeSequence를 추출하여 해당 파일을 바탕으로 악성코드를 탐지하는 함수를 제작하였다.

In [11]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunctionUsingPickle(model, pickleName):
    with open(pickleName, 'rb') as lf:
        sampleOpcodeList = pickle.load(lf)

    sampleFrequencyList = [0 for i in range(len(opcodes))]
    for op in sampleOpcodeList:
        sampleFrequencyList[opcodeDict[op]] += 1

    dict_list = [[] for o in opcodes]

    for i in range(len(opcodes)):
        dict_list[i].append(sampleFrequencyList[i])

    dictionary = dict(zip(opcodes, dict_list))
    sampleDf = pd.DataFrame.from_dict(dictionary)
    sampleDf.head()

    return model.predict(sampleDf)[0]

- 악성코드 파일이기 때문에 모델이 1로 분류하는 것을 확인할 수 있다.

In [12]:
pickleName = 'MalwareSample.p'

MalwareDetectionFunctionUsingPickle(clf, pickleName)

1